# Classification de tweets avec un LLM (Ollama + Mistral)

Dans ce notebook, on va :

1. Charger un fichier CSV de tweets nettoyés.
2. Préparer un prompt avec quelques exemples (few-shot).
3. Appeler un modèle LLM local (Ollama, modèle *mistral*) pour classer chaque tweet.
4. Récupérer la réponse du modèle en JSON et la parser proprement.
5. Normaliser les résultats (topics, sentiment, incident, etc.).
6. Fusionner les prédictions avec le CSV d’origine et sauvegarder le résultat.

Ce notebook reprend la logique du script Python, mais de manière plus pédagogique et étape par étape.


In [ ]:
# Imports des bibliothèques nécessaires

import requests          # pour appeler l'API HTTP d'Ollama
import pandas as pd      # pour lire/écrire les CSV
import orjson           # pour parser le JSON renvoyé par le modèle
import time             # pour faire des pauses entre les retries
import sys              # pour quitter proprement (utile si fichier manquant)
from pathlib import Path # pour travailler proprement avec les chemins de fichiers

# ====== CONFIG GENERALE ======

MODEL = "mistral"                                 # Nom du modèle Ollama
SOURCE_CSV = "../data/free_tweet_export.cleaned.llm.csv"  # Fichier des tweets nettoyés
OUTPUT_CSV = "../data/free_tweet_classified_clean.csv"    # Fichier de sortie avec les prédictions
TEXT_COL = "text_clean_llm"                       # Nom de la colonne texte dans le CSV

# Paramètres pour éventuellement limiter le nombre de lignes (pour debug) :
SAMPLE_N = None        # par ex: 200 pour tester uniquement sur 200 lignes
SAVE_SAMPLE_AS = None  # ex: "echantillon.csv" pour sauvegarder l'échantillon

# Paramètres d'appel à l'API
API_URL = "http://localhost:11434/api/chat"
BATCH_SIZE = 20        # nombre de tweets à traiter par batch
HTTP_TIMEOUT = 300     # timeout HTTP en secondes
MAX_RETRIES = 3        # nombre de tentatives en cas d'erreur
BACKOFFS = [1, 3, 7]   # temps d'attente entre tentatives

# Session HTTP réutilisée pour optimiser les appels réseau
SESSION = requests.Session()


## Chargement des données

Dans cette partie, on va :

- Vérifier que le fichier CSV source existe.
- Charger le CSV dans un DataFrame `df_full`.
- Vérifier que la colonne texte choisie (`TEXT_COL`) est bien présente.
- (Optionnel) Prendre seulement un échantillon des données si `SAMPLE_N` est défini.


In [ ]:
# Vérification de l'existence du fichier
if not Path(SOURCE_CSV).exists():
    print(f"Fichier introuvable: {SOURCE_CSV}")
    sys.exit(1)

# Lecture du fichier CSV complet
df_full = pd.read_csv(SOURCE_CSV, engine="python")
print(f"Colonnes trouvées dans le CSV : {list(df_full.columns)}")

# Vérification de la présence de la colonne texte
if TEXT_COL not in df_full.columns:
    raise SystemExit(f"Colonne {TEXT_COL} introuvable dans {SOURCE_CSV}")

# Gestion de l'échantillonnage
if SAMPLE_N is not None:
    df = df_full.head(int(SAMPLE_N)).copy()
    if SAVE_SAMPLE_AS:
        df.to_csv(SAVE_SAMPLE_AS, index=False, encoding="utf-8")
        print(f"💾 Échantillon sauvegardé dans {SAVE_SAMPLE_AS} ({len(df)} lignes)")
    else:
        print(f"🔎 Échantillon en mémoire: {len(df)} lignes (non sauvegardé)")
else:
    df = df_full.copy()
    print(f"🔎 Traitement de tout le fichier: {len(df)} lignes")

df.head()


## Définition du prompt et des exemples (few-shot)

Ici, on prépare :

- Le *prompt système* (contexte global) lu depuis un fichier `prompt_tweets_free.txt`.
- Un template de message utilisateur qui contient le tweet et un exemple de JSON attendu.
- Une liste d'exemples `FEW_SHOTS` (tweet → réponse JSON) pour montrer au modèle le format voulu.
- La variable `BASE_MESSAGES` qui contient :
  - le message système
  - les exemples few-shot

Cette base sera réutilisée pour chaque appel au modèle.


In [ ]:
# Lecture du prompt système depuis un fichier texte
with open("prompt_tweets_free.txt", "r", encoding="utf-8") as f:
    SYSTEM_PROMPT = f.read()

# Template du message utilisateur (on y insérera chaque tweet)
USER_TEMPLATE = """Tweet:
{tweet}

Réponds en JSON (une seule ligne), exemple:
{{"is_claim":1,"topics":["fibre"],"sentiment":"neg","urgence":"haute","incident":"incident_reseau","confidence":0.82}}"""

# Exemples few-shot (tweet -> JSON attendu)
FEW_SHOTS = [ 
    ("rt @free: découvrez la nouvelle chaîne imearth en 4k !",
     '{"is_claim":0,"topics":["tv"],"sentiment":"neu","urgence":"basse","incident":"information","confidence":0.9}'),
    ("@free panne fibre à cergy depuis 7h, impossible de bosser",
     '{"is_claim":1,"topics":["fibre"],"sentiment":"neg","urgence":"haute","incident":"incident_reseau","confidence":0.9}'),
    ("@freebox non mais vous répondez trois jours après... super le service après vente",
     '{"is_claim":1,"topics":["autre"],"sentiment":"neg","urgence":"moyenne","incident":"processus_sav","confidence":0.85}')
]

# Préparation de la base de messages commune : system + few-shot
BASE_MESSAGES = [{"role": "system", "content": SYSTEM_PROMPT}]
for u_text, y_json in FEW_SHOTS:
    BASE_MESSAGES.append({"role": "user", "content": u_text})
    BASE_MESSAGES.append({"role": "assistant", "content": y_json})

print("Prompt système et few-shots initialisés.")


## Fonctions utilitaires

Nous allons définir trois fonctions importantes :

1. `chat_ollama_batch(model, prompts)`  
   → envoie les prompts au modèle Ollama (un par un), avec gestion des erreurs et retries.

2. `_norm_incident(x)`  
   → normalise la variable `incident` pour la ramener à un petit set de valeurs autorisées
   (ex. "réseau", "sav", "support" → catégories standard).

3. `parse_json_line(s)`  
   → essaie de parser proprement la réponse du LLM en JSON,
   même si la réponse est entourée de ```json ... ``` ou contient du texte en plus.


In [ ]:
def chat_ollama_batch(model: str, prompts: list[str]) -> list[str]:
    """
    Appelle Ollama (API chat) de manière séquentielle avec retries/backoff.
    - `prompts` : liste de textes (un par tweet)
    - Retour : liste de réponses brutes (strings), une par prompt
    """
    out: list[str] = []

    for p in prompts:
        # messages = system + few-shots + message utilisateur
        messages = BASE_MESSAGES + [{"role": "user", "content": p}]
        payload = {
            "model": model,
            "messages": messages,
            "stream": False,
            "options": {"temperature": 0.1}
        }

        last_err = None
        for attempt in range(1, MAX_RETRIES + 1):
            try:
                r = SESSION.post(API_URL, json=payload, timeout=HTTP_TIMEOUT)
                r.raise_for_status()
                data = r.json()
                content = data.get("message", {}).get("content", "").strip()
                if content:
                    out.append(content)
                    break
                else:
                    last_err = "empty_content"
                    raise RuntimeError("Empty content from model")
            except Exception as e:
                last_err = str(e)
                # si erreur, on attend un peu avant de retenter
                if attempt < MAX_RETRIES:
                    time.sleep(BACKOFFS[min(attempt-1, len(BACKOFFS)-1)])
                else:
                    # si toutes les tentatives échouent, on renvoie un JSON d'erreur minimal
                    safe_err = last_err.replace('"', "'") if last_err else "unknown_error"
                    out.append(
                        '{"is_claim":0,"topics":["autre"],"sentiment":"neu","urgence":"basse","incident":"autre","confidence":0.0,"_error":"%s"}'
                        % safe_err
                    )
    return out


def _norm_incident(x: str) -> str:
    """
    Normalise la valeur 'incident' vers un set de valeurs autorisées,
    en gérant quelques alias courants.
    """
    allowed = {"facturation", "incident_reseau", "livraison", "information", "processus_sav", "autre"}
    s = str(x).strip().lower().replace(" ", "_")

    aliases = {
        "reseau": "incident_reseau",
        "réseau": "incident_reseau",
        "incident": "incident_reseau",
        "sav": "processus_sav",
        "service_apres_vente": "processus_sav",
        "service_après_vente": "processus_sav",
        "support": "processus_sav",
        "facture": "facturation",
        "info": "information",
        "livraisons": "livraison"
    }

    s = aliases.get(s, s)
    return s if s in allowed else "autre"


def parse_json_line(s: str) -> dict:
    """
    Parse robuste de la ligne JSON renvoyée par le LLM.
    - Gère les blocs ```json ... ```
    - Essaie d'extraire la partie entre { et }
    - Renvoie un dict par défaut si tout échoue
    """
    s = str(s).strip()

    # Gestion des blocs ```json ... ```
    if s.startswith("```"):
        lines = s.splitlines()
        if len(lines) >= 3 and lines[0].startswith("```") and lines[-1].startswith("```"):
            s = "\n".join(lines[1:-1]).strip()
        else:
            # fallback : on enlève les backticks et un éventuel 'json'
            s = s.strip("`").strip()
            if s.lower().startswith("json"):
                s = s[4:].strip()

    # Première tentative : parse direct
    try:
        return orjson.loads(s)
    except Exception:
        pass

    # Deuxième tentative : on isole le premier { et le dernier }
    try:
        start = s.find("{")
        end = s.rfind("}")
        if start != -1 and end != -1 and end > start:
            return orjson.loads(s[start:end+1])
    except Exception:
        pass

    # Fallback : objet par défaut
    return {
        "is_claim": 0,
        "topics": ["autre"],
        "sentiment": "neu",
        "urgence": "basse",
        "incident": "autre",
        "confidence": 0.0,
        "_parse_error": True
    }

print("Fonctions utilitaires prêtes.")
